In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from CRUDDriverMongoDB import CRUDDriverMongoDB

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = CRUDDriverMongoDB(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read('animals'))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso Salvare Logo.png'

with open(image_filename, 'rb') as f:
    encoded_image = base64.b64encode(f.read()).decode('utf-8')
    

app.layout = html.Div([
    html.Div(
        style={
            'display': 'flex', 
            'alignItems': 'center',
            'justifyContent': 'center',
            'marginBottom': '10px'
        },
        
        children=[
            html.A(
                href='https://www.snhu.edu',
                target="_blank",
                children=[
                    html.Img(
                        src=f'data:image/png;base64,{encoded_image}',
                        style={
                            'height': '80px',
                            'marginRight': '20px'
                        }
                    )
                ],
            ),
            html.Div(
                children=[
                    html.H1('CS-340 Dashboard',
                            style={'margin': '0px', 'padding': '0px'}),
                    html.H3('Unique Identifier: CS340 - Zane Milo Deso',
                            id="unique-id", 
                            style={'margin': '0px', 'fontWeight': 'normal'})
                ],
            )
        ],
    ),
    html.Hr(),

        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster/Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block'}
    ),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),

                        #________________________#
                        #       DATA TABLE       #
                        #________________________#

                         page_size=10,
                         filter_action="native",
                         sort_action="native",
                         row_selectable="single",
                         selected_rows=[0],
                         style_table={'overflowX': 'auto'},
                         style_cell={
                            'textAlign': 'left',
                            'padding': '5px'
                         },
                         style_header={
                            'backgroundColor': 'lightgrey',
                            'fontWeight': 'bold'
                         },

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

     # Start with a copy of the full data
    dff = df.copy()

    if filter_type == 'water':
        # Filter to only water rescue (example: by breed, age, etc.)
        # Suppose you have a 'breed' column or 'rescue_type' column.
        # You can also combine breed + sex + age logic if needed.
        dff = dff[
            (dff['breed'].isin(['Labrador Retriever Mix',
                                'Chesapeake Bay Retriever',
                                'Newfoundland'])) &
            (dff['sex_upon_outcome'] == 'Intact Female') &
            (dff['age_upon_outcome_in_weeks'] >= 26) &
            (dff['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif filter_type == 'mountain':
        dff = dff[
            (dff['breed'].isin(['German Shepherd',
                                'Alaskan Malamute',
                                'Old English Sheepdog',
                                'Siberian Husky',
                                'Rottweiler'])) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'] >= 26) &
            (dff['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif filter_type == 'disaster':
        dff = dff[
            (dff['breed'].isin(['Doberman Pinscher',
                                'German Shepherd',
                                'Golden Retriever',
                                'Bloodhound',
                                'Rottweiler'])) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'] >= 20) &
            (dff['age_upon_outcome_in_weeks'] <= 300)
        ]
    elif filter_type == 'reset':
        # Return entire unfiltered data
        dff = df.copy()

    # Return the filtered rows for the data table
    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    return [dcc.Graph(figure=fig)]
 
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)
